In [1]:
# !pip install ruclip==0.0.2

In [2]:
!nvidia-smi

Thu Sep 22 12:56:43 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.47.03    Driver Version: 510.47.03    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  On   | 00000000:09:00.0 Off |                  N/A |
| 27%   38C    P8    18W / 250W |      6MiB / 11264MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  On   | 00000000:42:00.0 Off |                  N/A |
| 22%   

In [1]:
import pandas as pd
import numpy as np
import torch
import os
import ruclip

device = "cuda" if torch.cuda.is_available() else "cpu"
# name_ruclip = 'ruclip-vit-base-patch16-224' #first clip metrica=0.600590
name_ruclip = "ruclip-vit-large-patch14-336"
clip, processor = ruclip.load(name_ruclip, device=device)
predictor = ruclip.Predictor(clip, processor, device, bs=1)

In [2]:
#считываем датафрейм, добавляем столбик для эмбеддингов
test_images_path = "/home/docker_current/datasets/train"
df_test = pd.read_csv("/home/docker_current/datasets/test.csv")
df_test['text_features'] = None
df_test['object_img'] = None

# считываем тестовые изображения, делаем словарь с эмбеддингами
test_images = [i.split('.png')[0] for i in os.listdir("/home/docker_current/datasets/test")]
test_embed = {i:None for i in test_images}

df_test.head(3)

,id,description,text_features,object_img
0,486,Фотография. Елизавета Алексеевна Юманова. ПКМ...,None,None
1,813,Фотография. Заседание комитета комсомола мотор...,None,None
2,2980,"Фотография. День ""Саланга"". ПОКМ-18530/638 фо...",None,None


In [3]:
from PIL import Image

def get_image_features(key):
    sample_image_path = "/home/docker_current/datasets/test/" + str(key) + ".png"
    image = Image.open(sample_image_path)
    
    with torch.no_grad():
        image_features = predictor.get_image_latents([image]).cpu().detach().numpy()
    return image_features

def get_text_features(df, index_text):
    sample_text = [df['description'][index_text]]
    text_features = predictor.get_text_latents(sample_text).cpu().detach().numpy()
    return text_features 

cos = torch.nn.CosineSimilarity(dim=1, eps=1e-6)
def get_similarity(image_emb, text_emb):
    sim =cos(torch.Tensor(image_emb), 
                        torch.Tensor(text_emb))
    return sim

In [6]:
%%time
for ind_text in range(len(df_test)):
    df_test['text_features'][ind_text] = get_text_features(df_test, ind_text)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


CPU times: user 42 s, sys: 115 ms, total: 42.1 s
Wall time: 42.2 s


In [7]:
for name_image in test_embed.keys():
    test_embed[name_image] = get_image_features(name_image)

1it [00:00, 46.88it/s]
1it [00:00, 60.90it/s]
1it [00:00, 61.87it/s]
1it [00:00, 43.36it/s]
1it [00:00, 64.63it/s]
1it [00:00, 46.53it/s]
1it [00:00, 40.14it/s]
1it [00:00, 54.63it/s]
1it [00:00, 61.70it/s]
1it [00:00, 48.18it/s]
1it [00:00, 39.81it/s]
1it [00:00, 65.48it/s]
1it [00:00, 56.88it/s]
1it [00:00, 58.38it/s]
1it [00:00, 39.05it/s]
1it [00:00, 51.93it/s]
1it [00:00, 60.68it/s]
1it [00:00, 55.28it/s]
1it [00:00, 60.45it/s]
1it [00:00, 56.46it/s]
1it [00:00, 58.55it/s]
1it [00:00, 65.01it/s]
1it [00:00, 53.47it/s]
1it [00:00, 50.82it/s]
1it [00:00, 48.31it/s]
1it [00:00, 60.91it/s]
1it [00:00, 59.01it/s]
1it [00:00, 59.16it/s]
1it [00:00, 54.46it/s]
1it [00:00, 61.06it/s]
1it [00:00, 59.05it/s]
1it [00:00, 59.07it/s]
1it [00:00, 60.93it/s]
1it [00:00, 50.94it/s]
1it [00:00, 38.14it/s]
1it [00:00, 43.01it/s]
1it [00:00, 49.22it/s]
1it [00:00, 47.19it/s]
1it [00:00, 54.27it/s]
1it [00:00, 40.41it/s]
1it [00:00, 58.38it/s]
1it [00:00, 45.05it/s]
1it [00:00, 51.46it/s]
1it [00:00,

In [8]:
from copy import deepcopy
copy_test_embed = deepcopy(test_embed)

In [9]:
preds = []

# ind_text = 0
for ind_text in range(len(df_test)):

    sims = []
    for image_name in copy_test_embed.keys():
        sim = get_similarity(copy_test_embed[image_name], 
                                    df_test['text_features'][ind_text])
        sims.append(sim)

    sims = np.array(sims)
    ind_max = np.argmax(sims)
    match_image = list(copy_test_embed.keys())[ind_max]
    preds.append(match_image)
    del copy_test_embed[match_image]

In [10]:
submit = pd.read_csv('/home/docker_current/py_files/sample_solution.csv')
submit['object_img'] = preds
submit['object_img'] = submit['object_img'].astype(np.int64)
submit.to_csv('ruclip-vit-large-patch14-336.csv', index=False)

In [48]:
# ind = 39
# text_features = get_text_features(df_test, ind)
# image_feat = get_image_features(list(test_embed.keys())[ind])

# get_similarity(text_features, image_feat)